# **노래 가사 분석을 통한 인기차트 Top 10**
출처 : https://rahites.tistory.com/103


1. 1990년~2022년 년도별 인기 top30곡의 노래 가사를 크롤링
2. 노래 가사 데이터를 여러 가지 방법으로 텍스트 분석 진행
3. 텍스트 분석 진행한 결과 바탕으로 1990-2019년까지의 가사 데이터를 train, 2020-2022년의 가사 데이터를 test로 분류하여 각각의 feature data 생성
4. 머신러닝을 이용한 top10 여부 예측

##**1. 데이터 크롤링**

In [1]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.2/400.2 kB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.7 MB/s eta 0:00:00
Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [458 kB]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:5 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:6 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [860 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:8 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [980 kB]
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu 

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By

In [3]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')        # Head-less 설정
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(options=options)

In [4]:
import pandas as pd
import numpy as np
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
import re
from bs4 import BeautifulSoup

driver.get("https://www.melon.com/chart/age/index.htm?chartType=YE&chartGenre=KPOP&chartDate=2022")
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

def melon_collector(url):
  time.sleep(5)
  driver.get(url)
  html = driver.page_source
  soup = BeautifulSoup(html, 'html.parser')

  #제목 가져오기
  title = driver.find_elements(by = By.CLASS_NAME, value = 'ellipsis.rank01')
  title2=[]
  for i in title:
    title2.append(i.text)

  del title2[30:]

  #가수 가져오기
  singer = driver.find_elements(by = By.CLASS_NAME, value = 'ellipsis.rank02')
  singer2=[]
  for i in singer :
    singer2.append(i.text)

  del singer2[30:]

  #가사 가져오기
  song_info = soup.find_all('div', attrs = {'class' : 'ellipsis rank01'})

  #Top 30 추출
  songid = []
  for i in range(30):
    try :
      songid.append(re.sub(f'[^0-9]', '', song_info[i].find("a")["href"][43:]))# 곡 id 추출
    except :
      songid.append('')
      continue #1992년 먼지가 되어와 같이 멜론에서 현재 들을 수 없는 노래 제외
  lyrics=[]
  for i in songid:
    try :
      driver.get("https:///www.melon.com/song/detail.htm?songId="+i)
      time.sleep(2)
      lyric = driver.find_element(by=By.CLASS_NAME, value='lyric')
      lyrics.append(lyric.text)
    except: #가사가 없는 경우
      lyrics.append('')
      continue
  lyrics2=[]

  for i in lyrics:
    lyrics2.append(i.replace('\n',' '))

  df = pd.DataFrame({"제목": title2, "가수":singer2, "가사":lyrics2})

  #저장하기
  df.to_csv(f'멜론{start}.csv', index=False)

In [8]:
start = 1990
url = 'https://www.melon.com/chart/age/index.htm?chartType=YE&chartGenre=KPOP&chartDate='

while start<=2022:
  new_url = url+str(start)
  melon_collector(new_url)
  print(start,'완료')
  start+=1

1990 완료
1991 완료
1992 완료
1993 완료
1994 완료
1995 완료
1996 완료
1997 완료
1998 완료
1999 완료
2000 완료
2001 완료
2002 완료
2003 완료
2004 완료
2005 완료
2006 완료
2007 완료
2008 완료
2009 완료
2010 완료
2011 완료
2012 완료
2013 완료
2014 완료
2015 완료
2016 완료
2017 완료
2018 완료
2019 완료
2020 완료
2021 완료
2022 완료


## **2. 가사 텍스트 분석 & 피처 생성**

In [6]:
import pandas as pd
import numpy as np

In [13]:
melon_1990s = pd.concat(map(pd.read_csv, ['멜론1990.csv', '멜론1991.csv','멜론1992.csv','멜론1993.csv','멜론1994.csv','멜론1995.csv','멜론1996.csv','멜론1997.csv','멜론1998.csv','멜론1999.csv']),axis=0)
melon_2000s = pd.concat(map(pd.read_csv, ['멜론2000.csv', '멜론2001.csv','멜론2002.csv','멜론2003.csv','멜론2004.csv','멜론2005.csv','멜론2006.csv','멜론2007.csv','멜론2008.csv','멜론2009.csv']),axis=0)
melon_2010s = pd.concat(map(pd.read_csv, ['멜론2010.csv', '멜론2011.csv','멜론2012.csv','멜론2013.csv','멜론2014.csv','멜론2015.csv','멜론2016.csv','멜론2017.csv','멜론2018.csv','멜론2019.csv']),axis=0)
melon_2020s = pd.concat(map(pd.read_csv, ['멜론2020.csv', '멜론2021.csv','멜론2022.csv']),axis=0)

In [14]:
melon_1990s.head()

,제목,가수,가사
0,희망사항,변진섭,청바지가 잘 어울리는 여자 밥을 많이 먹어도 배 안나오는 여자 내 얘기가 재미 없어...
1,사랑일뿐야,김민우,나를 어떻게 생각하냐고 너는 내게 묻지만 대답하기는 힘들어 너에게 이런 얘길 한다면...
2,유리창엔 비,햇빛촌,낮부터 내린 비는 이 저녁 유리창에 이슬만 뿌려놓고서 밤이 되면 더욱 커지는 시계소...
3,비오는 날 수채화,"김현식, 강인원, 권인하",빗방울 떨어지는 그 거리에 서서 그대 숨소리 살아있는 듯 느껴지면 깨끗한 붓 하나를...
4,마지막 콘서트,이승철,지금 슬픈 내 모습은 무대뒤 한 소녀 애써 눈물 참으며 바라보고 있네 무대 뒤에 그...


In [15]:
melon_total = [melon_1990s, melon_2000s, melon_2010s, melon_2020s]

In [17]:
#Top10인지 아닌지 여부 column
for melon_year in melon_total :
  melon_year['Top10'] = 0

for melon_year in melon_total :
  for i in melon_year.index:
    if i<=10:
      melon_year['Top10'][i] +=1
    else:
      continue

<ipython-input-17-1156408976d1>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  melon_year['Top10'][i] +=1


In [18]:
!pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 35.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 25.2 MB/s eta 0:00:00


In [19]:
#가사 column 따로 지정
gasa_1990s = melon_1990s.가사
gasa_2000s = melon_2000s.가사
gasa_2010s = melon_2010s.가사
gasa_2020s = melon_2020s.가사

#Komoran
from konlpy.tag import Komoran

Komoran = Komoran()

noun2_1990s = []
noun2_2000s = []
noun2_2010s = []
noun2_2020s = []

for i in gasa_1990s:
  try :
    noun2_1990s.append(Komoran.nouns(i))
  except:
    continue

for i in gasa_2000s:
  try :
    noun2_2000s.append(Komoran.nouns(i))
  except:
    continue

for i in gasa_2010s:
  try :
    noun2_2010s.append(Komoran.nouns(i))
  except:
    continue

for i in gasa_2020s:
  try :
    noun2_2020s.append(Komoran.nouns(i))
  except:
    continue

In [20]:
#한국어 불용어
with open('/content/drive/MyDrive/ESAA/stopword.txt','r', encoding='utf-8') as f:
  text = f.read()
stopwords = text.split()

#불용어 처리
noun2_1990s_sw = [];
noun2_2000s_sw = [];
noun2_2010s_sw = [];
noun2_2020s_sw = [];

#2
for text in noun2_1990s:
  temp = []
  for word in text :
    if word not in stopwords :
      temp.append(word)
  noun2_1990s_sw.append(temp)

for text in noun2_2000s:
  temp = []
  for word in text :
    if word not in stopwords :
      temp.append(word)
  noun2_2000s_sw.append(temp)

for text in noun2_2010s:
  temp = []
  for word in text :
    if word not in stopwords :
      temp.append(word)
  noun2_2010s_sw.append(temp)

for text in noun2_2020s:
  temp = []
  for word in text :
    if word not in stopwords :
      temp.append(word)
  noun2_2020s_sw.append(temp)

###2.1 빈도 분석

In [21]:
#Top 30 노래 가사에서 등장한 단어의 빈도
#Komoran
#가장 많이 등장한 Top 10 단어들의 빈도를 Feature로 사용

from collections import Counter

mostwords_1990s = []
mostwords_2000s = []
mostwords_2010s = []
mostwords_2020s = []

for document in noun2_1990s_sw:
  mostwords_1990s.append([y for x,y in Counter(document).most_common(10)])
for document in noun2_2000s_sw:
  mostwords_2000s.append([y for x,y in Counter(document).most_common(10)])
for document in noun2_2010s_sw:
  mostwords_2010s.append([y for x,y in Counter(document).most_common(10)])
for document in noun2_2020s_sw:
  mostwords_2020s.append([y for x,y in Counter(document).most_common(10)])

#DataFrame 형태로 변경
mostwords_1990s = pd.DataFrame(mostwords_1990s).fillna(0)
mostwords_1990s.columns = mostwords_1990s.columns.map(lambda x : '빈도_'+str(x+1)+'등')
mostwords_2000s = pd.DataFrame(mostwords_2000s).fillna(0)
mostwords_2000s.columns = mostwords_2000s.columns.map(lambda x : '빈도_'+str(x+1)+'등')
mostwords_2010s = pd.DataFrame(mostwords_2010s).fillna(0)
mostwords_2010s.columns = mostwords_2010s.columns.map(lambda x : '빈도_'+str(x+1)+'등')
mostwords_2020s = pd.DataFrame(mostwords_2020s).fillna(0)
mostwords_2020s.columns = mostwords_2020s.columns.map(lambda x : '빈도_'+str(x+1)+'등')

#melon dataframe에 가사가 없는 행 제거
melon_1990s.dropna(inplace=True)
melon_2000s.dropna(inplace=True)
melon_2010s.dropna(inplace=True)
melon_2020s.dropna(inplace=True)

In [22]:
feature = []
feature_te = []

# train
bindo_train = pd.concat([mostwords_1990s, mostwords_2000s, mostwords_2010s], axis=0).reset_index(drop=True)
feature.append(bindo_train)

# test
feature_te.append(mostwords_2020s)

### **2-2. TF-IDF**

In [23]:
tf_noun_1990s = []
tf_noun_2000s = []
tf_noun_2010s = []
tf_noun_2020s = []

for i in noun2_1990s_sw:
    tf_noun_1990s.append(" ".join(i))
for i in noun2_2000s_sw:
    tf_noun_2000s.append(" ".join(i))
for i in noun2_2010s_sw:
    tf_noun_2010s.append(" ".join(i))
for i in noun2_2020s_sw:
    tf_noun_2020s.append(" ".join(i))

In [27]:
# 1990년대에 주로 사용된 단어 Top 10으로
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer_1990s = TfidfVectorizer(max_features =10)  # 1글자 사용x, L2 정규화
vectorizer_2000s = TfidfVectorizer()
vectorizer_2010s = TfidfVectorizer()
vectorizer_2020s = TfidfVectorizer()

doc_term_mat_1990s_90 = vectorizer_1990s.fit_transform(tf_noun_1990s)
doc_term_mat_d_1990s_90 = doc_term_mat_1990s_90.toarray() # 희소행렬

doc_term_mat_2000s_90 = vectorizer_1990s.transform(tf_noun_2000s)
doc_term_mat_d_2000s_90 = doc_term_mat_2000s_90.toarray()

doc_term_mat_2010s_90 = vectorizer_1990s.transform(tf_noun_2010s)
doc_term_mat_d_2010s_90 = doc_term_mat_2010s_90.toarray()

doc_term_mat_2020s_90 = vectorizer_1990s.transform(tf_noun_2020s)
doc_term_mat_d_2020s_90 = doc_term_mat_2020s_90.toarray()

tfidf_1990s = pd.DataFrame(doc_term_mat_d_1990s_90)
tfidf_2000s = pd.DataFrame(doc_term_mat_d_2000s_90)
tfidf_2010s = pd.DataFrame(doc_term_mat_d_2010s_90)
tfidf_2020s = pd.DataFrame(doc_term_mat_d_2020s_90)
tfidf_1990s.columns = vectorizer_1990s.get_feature_names_out()
tfidf_1990s.columns = tfidf_1990s.columns.map(lambda x : '1990_' + str(x))
tfidf_2000s.columns = tfidf_2000s.columns.map(lambda x : '1990_' + str(x))
tfidf_2010s.columns = tfidf_2010s.columns.map(lambda x : '1990_' + str(x))
tfidf_2020s.columns = tfidf_2020s.columns.map(lambda x : '1990_' + str(x))

In [28]:
tfidf_1990s

,1990_기억,1990_눈물,1990_마음,1990_모습,1990_사람,1990_사랑,1990_생각,1990_세상,1990_시간,1990_이제
0,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.563260,0.826280,0.000000,0.000000,0.000000
2,0.796038,0.000000,0.396109,0.000000,0.000000,0.000000,0.000000,0.000000,0.238075,0.390822
3,0.000000,0.000000,0.000000,0.000000,0.939112,0.000000,0.000000,0.343611,0.000000,0.000000
4,0.000000,0.453805,0.000000,0.427143,0.000000,0.000000,0.000000,0.000000,0.000000,0.782055
...,...,...,...,...,...,...,...,...,...,...
290,0.000000,0.000000,0.770079,0.138329,0.139007,0.098636,0.578783,0.000000,0.154281,0.000000
291,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
292,0.000000,0.000000,0.151919,0.000000,0.000000,0.233505,0.171271,0.000000,0.730469,0.599565
293,0.403872,0.000000,0.000000,0.000000,0.000000,0.695009,0.000000,0.000000,0.000000,0.594853


In [29]:
#TF-IDF 분석 결과도 피처로 활용하기 위해 10개의 column을 추가합니다.
tfidf_90_train = pd.concat([tfidf_1990s, tfidf_2000s, tfidf_2010s], axis=0).reset_index(drop=True)
feature.append(tfidf_90_train)

# test
feature_te.append(tfidf_2020s)

### **2-3. 토픽모델링**


In [32]:
pip install gensim

In [42]:
from gensim import models
from gensim.models import LdaModel

In [ ]:
# Perplexity(혼란도) 확률 모델이 결과를 얼마나 정확하게 예측하는지.낮을수록 정확하게 예측
# Coherence Score 을 판단, 토픽이 얼마나 의미론적으로 일관성 있는지, 높을수록 의미론적 일관성 높음

TOPICS_W_NUM =20 # 출력할 토픽별 단어의 개수
save_lda_model=0
RANDOM_STATE = 2020
UPDATE_EVERY = 1
CHUNKSIZE = 100
PASSES = 10
ALPHA = 'auto'
PER_WORD_TOPICS = True
print('NUM_TOPICS', 'perplexity', 'coherence')

for i in range(1,30):
    NUM_TOPICS=i

    lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=id2word,
                                              num_topics=NUM_TOPICS, random_state=RANDOM_STATE,
                                              update_every=UPDATE_EVERY, chunksize=CHUNKSIZE,
                                              passes=PASSES, alpha=ALPHA, per_word_topics=PER_WORD_TOPICS)

    doc_lda = lda_model[corpus]

    # Coherence Score
    coherence_model_lda = CoherenceModel(model=lda_model, texts=result_data, dictionary=id2word, coherence='c_v')
    coherence_lda = coherence_model_lda.get_coherence()

print('T',NUM_TOPICS, lda_model.log_perplexity(corpus), coherence_lda)

In [ ]:
# 시각화
# pyLDAvis.enable_notebook() # 주피터 노트북 실행시
def create_vis(model):
    pyLDAvis.enable_notebook()
    vis = pyLDAvis.gensim_models.prepare(model, corpus, id2word, sort_topics=False)
    # pyLDAvis.save_html(vis, './Result_lda_vis.html')
    return vis

create_vis(lda_model12) # 12개로 토픽 모델링한 모델

###**2-4. W2V(Word2Vector)**

In [33]:
 from gensim.models import word2vec

In [ ]:
w2v_90 = word2vec.Word2Vec(sentences = w2v_input_90,
                        vector_size = 30,
                        window = 3,
                        min_count = 1,
                        sg = 1).wv
w2v_00 = word2vec.Word2Vec(sentences = w2v_input_00,
                        vector_size = 30,
                        window = 3,
                        min_count = 1,
                        sg = 1).wv
w2v_10 = word2vec.Word2Vec(sentences = w2v_input_10,
                        vector_size = 30,
                        window = 3,
                        min_count = 1,
                        sg = 1).wv
w2v_20 = word2vec.Word2Vec(sentences = w2v_input_20,
                        vector_size = 30,
                        window = 3,
                        min_count = 1,
                        sg = 1).wv

In [ ]:
# train
w2v_train = pd.concat([train_w2v_90, train_w2v_00, train_w2v_10], axis=0).reset_index(drop=True)
w2v_train.columns = w2v_train.columns.map(lambda x : 'w2v_' + str(x))
feature.append(w2v_train)

# test
train_w2v_20.columns = train_w2v_20.columns.map(lambda x : 'w2v_' + str(x))
feature_te.append(train_w2v_20)


In [ ]:
# X_train
melon_train = pd.concat([melon_1990s,melon_2000s, melon_2010s], axis=0)
data = pd.DataFrame(melon_train.제목).reset_index(drop=True)

for f in feature :
    data = pd.concat([data, f], axis = 1)

data = data.fillna(round(data.mean(),4)) # 평균으로 결측치 채우기

# X_test
melon_test = melon_2020s
data_te = pd.DataFrame(melon_test.제목).reset_index(drop=True)

for f_te in feature_te :
    data_te = pd.concat([data_te, f_te], axis = 1)

data_te = data_te.fillna(round(data_te.mean(),4))

# y_train
y_train = melon_train.reset_index(drop=True).Top10

# y_test
y_test = melon_test.Top10

##**3. 모델링**

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import ExtraTreesClassifier
from lightgbm import LGBMClassifier

from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=4, shuffle=True, random_state=2020)
from sklearn.model_selection import train_test_split
X_train2, X_dev, y_train2, y_dev = train_test_split(train, y_train, test_size=0.3, random_state=2020, shuffle=True, stratify=y_train)

lgbm = LGBMClassifier(random_state=2020,n_jobs= -1)
rf = RandomForestClassifier(random_state = 2020)
lr = LogisticRegression(random_state=2020,n_jobs= -1)
knn = KNeighborsClassifier(n_jobs= -1)
et = ExtraTreesClassifier(random_state=2020,n_jobs= -1)

lgbm.fit(X_train2, y_train2)
rf.fit(X_train2, y_train2)
lr.fit(X_train2, y_train2)
knn.fit(X_train2, y_train2)
et.fit(X_train2, y_train2)

# 튜닝하지 않고 기본 모델의 결과 확인
# accuracy의 경우 전부 0으로 예측하더라도 0.667 score가 나오기 때문에 ROC-AUC 평가지표를 활용
from sklearn.model_selection import cross_val_score
print('#### 만들어진 피처의 결과는....? ####')
cv_score1 = cross_val_score(lgbm, X_dev, y_dev, cv=skf, scoring = 'roc_auc')
print('lgbm : ',cv_score1)
print('평균 : ',np.mean(cv_score1))
cv_score2 = cross_val_score(rf, X_dev, y_dev, cv=skf, scoring = 'roc_auc')
print('rf : ',cv_score2)
print('평균 : ',np.mean(cv_score2))
cv_score3 = cross_val_score(lr, X_dev, y_dev, cv=skf, scoring = 'roc_auc')
print('lr : ',cv_score3)
print('평균 : ',np.mean(cv_score3))
cv_score4 = cross_val_score(knn, X_dev, y_dev, cv=skf, scoring = 'roc_auc')
print('knn : ',cv_score4)
print('평균 : ',np.mean(cv_score4))
cv_score5 = cross_val_score(et, X_dev, y_dev, cv=skf, scoring = 'roc_auc')
print('et : ',cv_score5)
print('평균 : ',np.mean(cv_score5))